In [1]:
import numpy as np
from Robot import UR5Arm
from fwdkin import fwdkin
from invkin import invkin
import math

In [2]:
ex = np.array([1,0,0])
ey = np.array([0,1,0])
ez = np.array([0,0,1])
l0 = 0.061 # base to servo 1
l1 = 0.0435 # servo 1 to servo 2
l2 = 0.08285 # servo 2 to servo 3
l3 = 0.08285 # servo 3 to servo 4
l4 = 0.07385 # servo 4 to servo 5
l5 = 0.05457 # servo 5 to gripper
P01 = ( l0 + l1 ) * ez 
P12 = np.zeros (3) # translation between 1 and 2 frame in 1 frame
P23 = l2 * ex # translation between 2 and 3 frame in 2 frame
P34 = - l3 * ez # translation between 3 and 4 frame in 3 frame
P45 = np.zeros (3) # translation between 4 and 5 frame in 4 frame
P5T = -( l4 + l5 ) * ex 
print(P01,P12,P23,P34,P45,P5T)

[0.     0.     0.1045] [0. 0. 0.] [0.08285 0.      0.     ] [-0.      -0.      -0.08285] [0. 0. 0.] [-0.12842 -0.      -0.     ]


In [3]:
P = np.array([P01, P12, P23, P34, P45, P5T]).T
H = np.array([ez, -ey, -ey, -ey, -ex]).T
print(P,H)
limits = None 
dofbot = UR5Arm(P, H, limits)

[[ 0.       0.       0.08285 -0.       0.      -0.12842]
 [ 0.       0.       0.      -0.       0.      -0.     ]
 [ 0.1045   0.       0.      -0.08285  0.      -0.     ]] [[ 0  0  0  0 -1]
 [ 0 -1 -1 -1  0]
 [ 1  0  0  0  0]]


In [4]:
theta = np.array([90,90,90,90,90])[None].T*math.pi/180
print(theta)
print(theta.shape)
T = np.around(fwdkin(dofbot, theta), 4)
print(T)

[[1.57079633]
 [1.57079633]
 [1.57079633]
 [1.57079633]
 [1.57079633]]
(5, 1)
[[-0.     -0.     -1.      0.    ]
 [-0.     -1.      0.      0.    ]
 [-1.      0.     -0.      0.3986]
 [ 0.      0.      0.      1.    ]]


In [7]:
theta = np.array([0,45,135,45,135])[None].T*math.pi/180
T_0 = fwdkin(dofbot, theta)
Rd = T_0[:3, :3]
Pd = T_0[:3, -1]
print(Pd, Rd)
q0 = np.array([50,50,50,50,50])[None].T*math.pi/180
print(q0)
truth, rv = invkin(dofbot, Rd, Pd, q0)
print(rv)
print(f"before: \n{T_0}")
print(f"after: \n{np.around(fwdkin(dofbot, rv),3)}")


[0.14939045 0.         0.33674045] [[-0.70710678 -0.5        -0.5       ]
 [ 0.         -0.70710678  0.70710678]
 [-0.70710678  0.5         0.5       ]]
[[0.87266463]
 [0.87266463]
 [0.87266463]
 [0.87266463]
 [0.87266463]]
[[-0.5566704  -0.24620194 -0.79341204]
 [-0.66341395  0.70658796  0.24620194]
 [ 0.5         0.66341395 -0.5566704 ]] [0.15816513 0.18849386 0.11814353]
[-9.72819666e-05 -1.64943387e-05  1.87312419e-05  1.04041371e-05
  1.73327121e-05 -5.79741676e-06]
Inverse Kinematics Converged after 94 iterations
[[1.16014813e-04]
 [7.85273655e-01]
 [2.35626337e+00]
 [7.85470281e-01]
 [2.35605691e+00]]
before: 
[[-0.70710678 -0.5        -0.5         0.14939045]
 [ 0.         -0.70710678  0.70710678  0.        ]
 [-0.70710678  0.5         0.5         0.33674045]
 [ 0.          0.          0.          1.        ]]
after: 
[[-0.707 -0.5   -0.5    0.149]
 [-0.    -0.707  0.707  0.   ]
 [-0.707  0.5    0.5    0.337]
 [ 0.     0.     0.     1.   ]]
